# Text Normalization (Chinese)

- `text_normalizer_zh.py`
- Including functions for:
    - word-seg chinese texts
    - clean up texts by removing duplicate spaces and line breaks
    - remove incompatible weird characters

In [1]:
import unicodedata
import re
#from nltk.corpus import wordnet
#import collections
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import text_normalizer_zh as tnz
 

## Normalization Functions

In [2]:
# %load text_normalizer_zh.py
import unicodedata
import re
#from nltk.corpus import wordnet
#import collections
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
import requests 
import pandas as pd

## Normalize unicode characters
def remove_weird_chars(text):
#     ```
#     (NFKD) will apply the compatibility decomposition, i.e. 
#     replace all compatibility characters with their equivalents. 
#     ```
    text = unicodedata.normalize('NFKD', text).encode('utf-8', 'ignore').decode('utf-8', 'ignore')
    return text


## Remove duplicate spaces
def remove_extra_linebreaks(text):
    lines = text.split(r'\n+')
    return '\n'.join([re.sub(r'[\s]+',' ', l).strip() for l in lines if len(l)!=0])


def remove_extra_spaces(text):
    return re.sub("\\s+"," ", text).strip()

import jieba
jieba.set_dictionary('../../../RepositoryData/data/jiaba/dict.txt.jiebatw.txt')

## Word Segmentation
def seg(text, return_list = False):
    text_seg = jieba.cut(text)
    if return_list:
        out = [w for w in text_seg]
    else:
        out = ' '.join(text_seg)
    return out


def remove_symbols(text):
    text = re.sub('[\u0021-\u002f\u003a-\u0040\u005b-\u0060\u007b-\u007e\u00a1-\u00bf\u2000-\u206f\u2013-\u204a\u20a0-\u20bf\u2100-\u214f\u2150-\u218b\u2190-\u21ff\u2200-\u22ff\u2300-\u23ff\u2460-\u24ff\u2500-\u257f\u2580-\u259f\u25a0-\u25ff\u2600-\u26ff\u2e00-\u2e7f\u3000-\u303f\ufe50-\ufe6f\ufe30-\ufe4f\ufe10-\ufe1f\uff00-\uffef─◆╱]+','',text)
    return text

def remove_numbers(text):
    return re.sub('\\d+',"", text)

def remove_alphabets(text):
    return re.sub('[a-zA-Z]+','', text)

def normalize_corpus(corpus, is_remove_extra_linebreaks=True,
                    is_remove_weird_chars=True,
                    is_seg=True,
                    is_remove_symbols=True,
                    is_remove_numbers=True,
                    is_remove_alphabets=True,
                    is_return_list = False):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        
        if is_remove_extra_linebreaks:
            doc = remove_extra_linebreaks(doc)
            
        if is_remove_weird_chars:
            doc = remove_weird_chars(doc)
           
        if is_seg:
            doc=seg(doc, is_return_list)
            
        if is_remove_symbols:
            doc=remove_symbols(doc)
            
        if is_remove_alphabets:
            doc=remove_alphabets(doc)
            
        if is_remove_numbers:
            doc=remove_numbers(doc)
            
        normalized_corpus.append(remove_extra_spaces(doc))
        
    return normalized_corpus

## Extract an Article

- Grab the first article from Google news for demonstration

In [3]:
url = 'https://news.google.com/topics/CAAqJQgKIh9DQkFTRVFvSUwyMHZNRFptTXpJU0JYcG9MVlJYS0FBUAE?hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
r = requests.get(url)
web_content = r.text
soup = BeautifulSoup(web_content,'lxml')
title = soup.find_all('a', class_='DY5T1d')
first_art_link = title[0]['href'].replace('.','https://news.google.com',1)

#print(first_art_link)
art_request = requests.get(first_art_link)
art_request.encoding='utf8'
soup_art = BeautifulSoup(art_request.text,'lxml')

art_content = soup_art.find_all('p')
art_texts = [p.text for p in art_content]
print(art_texts)

['今年國慶焰火昨晚在台南登場，湧入逾50萬人遊客，交通接駁嚴重打結，不少人在網路開轟，台南市長黃偉哲下午受訪時表示，千錯萬錯，都由他承擔，將加強檢討、改進，未來還有大型活動時，疏散計畫列為第一優先。', '\n', '\r\n山上花園水道博物館第70萬名遊客下午出爐，黃偉哲前往頒獎給幸運兒，對媒體詢問，國慶焰火交通疏運網路上批評聲音，黃偉哲強調，「會自我檢討」。', '\n', '\r\n黃偉哲說，昨晚國慶焰火100分，但地上交通以及相關設施等，遊客來得太多，是預估的2倍以上，衝擊交通，只能打70分，市政府團隊還是要努力。', '\n', '\r\n黃偉哲指出，昨晚的活動，警察認真維持治安，沒有重大案件，交通指揮至深夜，相當辛苦；清潔方面，環保局的同仁與志工們一樣很努力，忙到今天凌晨4點多，大家都很用心。', '\n', '\r\n黃偉哲表示，千錯萬錯，都是市政府，要自己改進、檢討，未來舉辦大型活動，交通疏運計畫列為第一優先，完成最妥善部署，希望做得更好。', '\n                    昨天是雙十國慶日，不過有網友發現在Google搜尋「中華民國建立時間」，跳出來的時間竟然不是1912年1月1日，而是1949年12月7日，也就是中華民國政權遷往台灣的時間。                  ', '\n                    今年國慶焰火昨晚在台南登場，湧入逾50萬人遊客，交通接駁嚴重打結，不少人在網路開轟，台南市長黃偉哲下午受訪時表示，千錯萬...                  ', '\n                    前總統馬英九出席國民黨國慶升旗活動時，質疑民進黨真心誠意慶祝國慶嗎？新北市長侯友宜今天接受媒體訪問時說，中華民國「養我、...                  ', '\n                    國慶日剛過，蔡英文總統在臉書分享相關知識，提醒大家不要再稱禮賓人員是「金釵」。蔡總統說，今年國慶，大家可能都有看到穿著制...                  ', '\n                    在紐約僑界促成下，宣傳台灣觀光的形象廣告連兩年雙十節登上時報廣場大型螢幕。駐紐約辦事處長李光章形容，這為中華民國109歲...                  ', '\n 

- Normalized results:

In [4]:
tnz.normalize_corpus([' '.join(art_texts)])

Building prefix dict from /Users/Alvin/GoogleDrive/_MySyncDrive/RepositoryData/data/jiaba/dict.txt.jiebatw.txt ...


Loading model from cache /var/folders/n7/ltpzwx813c599nfxfb94s_640000gn/T/jieba.u1b52b47246a0f2e6497af6bbe107adac.cache


Loading model cost 0.603 seconds.


Prefix dict has been built successfully.


['今年 國慶 焰火 昨晚 在 台南 登場 湧入 逾 萬人 遊客 交通 接駁 嚴重 打結 不少 人 在 網路 開轟 台南 市長 黃偉哲 下午 受訪 時 表示 千錯萬錯 都 由 他 承擔 將 加強 檢討 改進 未來 還有 大型 活動 時 疏散 計畫 列為 第一 優先 山上 花園 水道 博物館 第 萬名 遊客 下午 出爐 黃偉哲 前往 頒獎給 幸運兒 對 媒體 詢問 國慶 焰火 交通 疏運 網路 上 批評 聲音 黃偉哲 強調 會 自我 檢討 黃偉哲 說 昨晚 國慶 焰火 分 但 地上 交通 以及 相關 設施 等 遊客 來得 太多 是 預估 的 倍 以上 衝擊 交通 只能 打 分 市政府 團隊 還是 要 努力 黃偉哲 指出 昨晚 的 活動 警察 認真 維持 治安 沒有 重大 案件 交通 指揮 至 深夜 相當 辛苦 清潔 方面 環保局 的 同仁 與 志工 們 一樣 很 努力 忙 到 今天 凌晨 點多 大家 都 很 用心 黃偉哲 表示 千錯萬錯 都 是 市政府 要 自己 改進 檢討 未來 舉辦 大型 活動 交通 疏運 計畫 列為 第一 優先 完成 最 妥善 部署 希望 做 得 更 好 昨天 是 雙十國慶日 不過 有 網友 發現 在 搜尋 中華民國 建立 時間 跳出來 的 時間 竟然 不是 年 月 日 而是 年 月 日 也 就是 中華民國 政權 遷往 台灣 的 時間 今年 國慶 焰火 昨晚 在 台南 登場 湧入 逾 萬人 遊客 交通 接駁 嚴重 打結 不少 人 在 網路 開轟 台南 市長 黃偉哲 下午 受訪 時 表示 千錯萬 前總統 馬英九 出席 國民黨 國慶 升旗 活動 時 質疑 民進黨 真心誠意 慶祝 國慶 嗎 新北 市長 侯友宜 今天 接受 媒體 訪問 時 說 中華民國 養 我 國慶日 剛過 蔡英文 總統 在 臉書 分享 相關 知識 提醒 大家 不要 再稱 禮賓 人員 是 金釵 蔡 總統 說 今年 國慶 大家 可能 都 有 看到 穿著 制 在 紐約 僑界 促成 下 宣傳 台灣 觀光 的 形象 廣告 連兩年 雙十節 登上 時報 廣場 大型 螢幕 駐 紐約 辦事處 長 李光章 形容 這為 中華民國 歲 台灣 基進 立委 陳 柏惟 昨說 雙十 正好 吃霜食 祝 大家 霜 食節 快樂 前立委 孫大千 在 臉書 表示 擔任 中華民國 立委 連句 生日快樂 都 不敢 波士頓 市政府 